### GROVER SIZE 8,16 IN IBM (Falcon) AND QUANTINUUM

In [1]:
# Caller
from pathlib import Path
import sys

# Discover project root
proj = Path.cwd().resolve()
for _ in range(8):
    if (proj / "results" / "pairwise" / "summary").exists() and (proj / "notebooks").exists():
        break
    if proj.parent == proj:
        break
    proj = proj.parent

# Import helper
sys.path.insert(0, str(proj / "notebooks"))
from notebook_helper import load_family, tidy, show_table  # (sort_best_worst, add_rank_within_size available if needed)

# analysis parameters
FAMILY = "grover"  # or: "qaoa","vqe_two_local","qft","grover","vbe_adder","randomcircuit"
BACKENDS = ("ibm_falcon","quantinuum")

loaded = load_family(FAMILY, BACKENDS)

for backend, df in loaded.items():
    tbl = tidy(df)
    show_table(tbl, caption=f"{FAMILY.upper()} – {backend} (raw tidy)")

    # To rank later, uncomment:
    # from notebook_helper import sort_best_worst, add_rank_within_size
    # PRIMARY_METRIC = "depth_delta"  # or "n_ops_total_gates_delta", "n_ops_total_delta", etc.
    # ranked = sort_best_worst(tbl, primary=PRIMARY_METRIC, per_size=True)
    # ranked = add_rank_within_size(ranked, by=PRIMARY_METRIC)
    # show_table(ranked, caption=f"{FAMILY.upper()} – {backend} (ranked by {PRIMARY_METRIC})")

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,ibm_falcon,grover,8,-,RB→RR,4995,4449,-546,7591,6687,-904,7582,6679,-903,5398,4495,-903,2192,2192,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 3439], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8]]",8,8,2025-10-05T22:28:50
1,ibm_falcon,grover,8,-,RR→RB,4995,4450,-545,7591,6694,-897,7582,6686,-896,5398,4502,-896,2192,2192,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 3446], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8]]",8,8,2025-10-05T22:28:50
2,ibm_falcon,grover,8,-,RB→CTM,4995,5112,+117,7591,7590,-1,7582,7582,+0,5398,5398,+0,2192,2192,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8]]",8,8,2025-10-05T22:28:51
3,ibm_falcon,grover,8,-,CTM→RB,4995,5112,+117,7591,7590,-1,7582,7582,+0,5398,5398,+0,2192,2192,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8]]",8,8,2025-10-05T22:28:51
4,ibm_falcon,grover,8,-,RR→CTM,4995,4567,-428,7591,6695,-896,7582,6686,-896,5398,4502,-896,2192,2192,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 3446], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-05T22:28:52
5,ibm_falcon,grover,8,-,CTM→RR,4995,4418,-577,7591,6392,-1199,7582,6383,-1199,5398,4199,-1199,2192,2192,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rz"", 4342], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 3143], [""cx"", 2192], [""sx"", 839], [""x"", 209], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-05T22:28:52
6,ibm_falcon,grover,16,-,RB→RR,636899,558795,-78104,852205,736459,-115746,852188,736443,-115745,617904,502159,-115745,234300,234300,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 494900], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 379155], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16]]",16,16,2025-10-05T22:30:28
7,ibm_falcon,grover,16,-,RR→RB,636899,558796,-78103,852205,736474,-115731,852188,736458,-115730,617904,502174,-115730,234300,234300,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 494900], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 379170], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16]]",16,16,2025-10-05T22:31:34
8,ibm_falcon,grover,16,-,RB→CTM,636899,655498,+18599,852205,852204,-1,852188,852188,+0,617904,617904,+0,234300,234300,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 494900], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 494900], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16]]",16,16,2025-10-05T22:32:48
9,ibm_falcon,grover,16,-,CTM→RB,636899,655498,+18599,852205,852204,-1,852188,852188,+0,617904,617904,+0,234300,234300,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 494900], [""cx"", 234300], [""sx"", 112479], [""x"", 10509], [""measure"", 16], [""barrier"", 1]]","[[""rz"

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,quantinuum,grover,16,-,RB→RR,1412960,1243259,-169701,2140982,1870187,-270795,2140965,1870171,-270794,1907107,1636313,-270794,233874,233874,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 579944], [""ry"", 578807], [""rz"", 477546], [""zzphase"", 233874], [""measure"", 16]]",16,16,2025-10-05T22:40:19
1,quantinuum,grover,16,-,RR→RB,1412960,1243259,-169701,2140982,1870187,-270795,2140965,1870171,-270794,1907107,1636313,-270794,233874,233874,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 579944], [""ry"", 578807], [""rz"", 477546], [""zzphase"", 233874], [""measure"", 16]]",16,16,2025-10-05T22:43:06
2,quantinuum,grover,16,-,RB→CTM,1412960,1577091,+164131,2140982,2140981,-1,2140965,2140965,+0,1907107,1907107,+0,233874,233874,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16]]",16,16,2025-10-05T22:46:15
3,quantinuum,grover,16,-,CTM→RB,1412960,1577091,+164131,2140982,2140981,-1,2140965,2140965,+0,1907107,1907107,+0,233874,233874,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16]]",16,16,2025-10-05T22:49:28
4,quantinuum,grover,16,-,RR→CTM,1412960,1385664,-27296,2140982,1870188,-270794,2140965,1870171,-270794,1907107,1636313,-270794,233874,233874,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 579944], [""ry"", 578807], [""rz"", 477546], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]",16,16,2025-10-05T22:52:29
5,quantinuum,grover,16,-,CTM→RR,1412960,1237155,-175805,2140982,1680334,-460648,2140965,1680317,-460648,1907107,1446459,-460648,233874,233874,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rx"", 825320], [""ry"", 579375], [""rz"", 502396], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]","[[""rx"", 579944], [""ry"", 578807], [""rz"", 287692], [""zzphase"", 233874], [""measure"", 16], [""barrier"", 1]]",16,16,2025-10-05T22:55:26
6,quantinuum,grover,8,-,RB→RR,12178,10565,-1613,19648,16815,-2833,19639,16807,-2832,17479,14647,-2832,2168,2168,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 7592], [""ry"", 5127], [""rz"", 4752], [""zzphase"", 2168], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 5160], [""ry"", 5095], [""rz"", 4384], [""zzphase"", 2168], [""measure"", 8]]",8,8,2025-10-05T22:55:29
7,quantinuum,grover,8,-,RR→RB,12178,10565,-1613,19648,16815,-2833,19639,16807,-2832,17479,14647,-2832,2168,2168,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 7592], [""ry"", 5127], [""rz"", 4752], [""zzphase"", 2168], [""measure"", 8], [""barrier"", 1]]","[[""rx"", 5160], [""ry"", 5095], [""rz"", 4384], [""zzphase"", 2168], [""measure"", 8]]",8,8,2025-10-05T22:55:30
8,quantinuum,grover,8,-,RB→CTM,12178,12993,+815,19648,19647,-1,19639,19639,+0,17479,17479,+0,2168,2168,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rx"", 7592], [""ry"", 5127], [""rz"", 4752], [""zzphase"", 2168], [""measure"", 8], [""barrier"", 1]]","[[""rx"",

### quick findings

- **CTM → RR** consistently looks like a strong reducer of depth across Grover runs. RR benefits a lot when CTM runs first (CTM exposes cancelable patterns that RR removes)
- **RB ↔ RR**: IBM shows tiny directional differences (diff = 1 in some sizes) whereas Quantinuum shows symmetric behavior, suggests the *native gateset / decomposition* or pass implementations are causing subtle differences.
- **RB ↔ CTM** tends to be neutral or harmful (depth increase).
- **Δs are dominated by 1-qubit gate reductions**: 2-qubit gate counts barely change -> passes are rearranging and canceling 1-qubit layers rather than affecting entangling structure.